<a href="https://colab.research.google.com/github/wondifraw/10Acadamy/blob/main/Scrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install telethon

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.0/722.0 kB 19.1 MB/s eta 0:00:00
  Created wheel for pyaes: filename=pyaes-1.6.1-py3-none-any.whl size=26347 sha256=732a3bbc7e797ca7c2600e0d396db40baf3051fe56c475352e7de1c7c66ce030
  Stored in directory: /root/.cache/pip/wheels/4e/52/33/010d0843550bffb6a591b11629070ae140c0ad4f53e68a3bd3
Successfully built pyaes


In [2]:
!pip install telethon python-dotenv

In [19]:
%%writefile telegram_scraper.py
from telethon import TelegramClient
from telethon.tl.types import MessageMediaPhoto, MessageMediaDocument
from typing import List, Dict
from telethon import TelegramClient
from telethon.tl.types import MessageMediaPhoto, MessageMediaDocument
import os
import json
from typing import List, Dict
import pandas as pd

class TelegramScraper:
    """
    Scrapes messages, images, and documents from specified Telegram channels.
    """
    def __init__(self, api_id: str, api_hash: str, session_name: str = 'anon'):
        """
        Initializes the scraper with Telegram API credentials.

        Args:
            api_id (str): Your Telegram API ID.
            api_hash (str): Your Telegram API hash.

        """
        self.api_id = api_id
        self.api_hash = api_hash
        self.session_name = session_name
        self.client = TelegramClient(session_name, api_id, api_hash)

    def get_channel_list(self) -> List[str]:
        """
        Returns a predefined list of Telegram channel usernames.

        Returns:
            List[str]: A list of channel usernames.
        """
        return [
            '@Leyueqa',
            '@sinayelj',
            '@Shewabrand',
            '@helloomarketethiopia',
            '@modernshoppingcenter',
        ]

    async def fetch_messages(self, channel: str, limit: int = 1000) -> List[Dict]:
        """
        Fetches messages from a given Telegram channel.

        Args:
            channel (str): The username or link of the Telegram channel.
            limit (int): The maximum number of messages to fetch.

        Returns:
            List[Dict]: A list of dictionaries, where each dictionary represents a message.
        """
        messages = []
        try:
            print(f"Connecting to Telegram...")
            async with self.client:
                print(f"Fetching channel entity for {channel}...")
                entity = await self.client.get_entity(channel)
                channel_title = getattr(entity, 'title', None)
                channel_username = getattr(entity, 'username', channel.strip('@'))
                print(f"Fetching messages from {channel}...")
                async for msg in self.client.iter_messages(channel, limit=limit):
                    media_path = None
                    if msg.media:
                        media_dir = os.path.join('data', 'raw', channel.strip('@'), 'media')
                        os.makedirs(media_dir, exist_ok=True)
                        try:
                            media_path = await msg.download_media(file=media_dir)
                        except Exception as e:
                            print(f"Could not download media for message {msg.id}: {e}")

                    msg_dict = {
                        'Channel Title': channel_title,
                        'Channel Username': channel_username,
                        'ID': msg.id,
                        'Message': msg.text,
                        'Date': str(msg.date),
                        'Media Path': media_path
                    }
                    messages.append(msg_dict)
            print(f"Successfully fetched {len(messages)} messages from {channel}.")
        except Exception as e:
            print(f"[ERROR] Failed to fetch messages from {channel}: {e}")
        return messages

    def save_data(self, data: List[Dict], out_path: str):
        """
        Saves data to a JSON file.

        Args:
            data (List[Dict]): The data to save.
            out_path (str): The path to the output JSON file.
        """
        try:
            os.makedirs(os.path.dirname(out_path), exist_ok=True)
            with open(out_path, 'w', encoding='utf-8') as f:
                json.dump(data, f, ensure_ascii=False, indent=2)
            print(f"Data successfully saved to {out_path}")
        except Exception as e:
            print(f"[ERROR] Failed to save data to {out_path}: {e}")

    def save_data_csv(self, data: List[Dict], out_path: str):
        """
        Saves data to a CSV file.

        Args:
            data (List[Dict]): The data to save.
            out_path (str): The path to the output CSV file.
        """
        try:
            os.makedirs(os.path.dirname(out_path), exist_ok=True)
            df = pd.DataFrame(data)
            df.to_csv(out_path, index=False, encoding='utf-8-sig')
            print(f"Data successfully saved to {out_path}")
        except Exception as e:
            print(f"[ERROR] Failed to save data to {out_path}: {e}")

Writing telegram_scraper.py


In [17]:
import sys
sys.path.append('/content/src')

In [22]:
from telegram_scraper import TelegramScraper
import asyncio

# Telegram API
api_id = '27932004'
api_hash = '611c1eca9d988b82f4008dc33126c6fe'

scraper = TelegramScraper(api_id, api_hash)
# Function to fetch messages from all channels and save as CSV
async def fetch_and_save_all(scraper, limit=1000):
    all_data = []
    for channel in scraper.get_channel_list():
        data = await scraper.fetch_messages(channel, limit=limit)
        all_data.extend(data)
    scraper.save_data_csv(all_data, 'data/raw/telegram_messages.csv')
    print('All messages saved to data/raw/telegram_messages.csv')
# Run the scraping and saving
await fetch_and_save_all(scraper, limit=1000)

Connecting to Telegram...
Fetching channel entity for @Leyueqa...
Fetching messages from @Leyueqa...
Successfully fetched 1000 messages from @Leyueqa.
Connecting to Telegram...
Fetching channel entity for @sinayelj...
Fetching messages from @sinayelj...
Successfully fetched 1000 messages from @sinayelj.
Connecting to Telegram...
Fetching channel entity for @Shewabrand...
Fetching messages from @Shewabrand...
Successfully fetched 1000 messages from @Shewabrand.
Connecting to Telegram...
Fetching channel entity for @helloomarketethiopia...
Fetching messages from @helloomarketethiopia...
Successfully fetched 1000 messages from @helloomarketethiopia.
Connecting to Telegram...
Fetching channel entity for @modernshoppingcenter...
Fetching messages from @modernshoppingcenter...
Successfully fetched 1000 messages from @modernshoppingcenter.
Data successfully saved to data/raw/telegram_messages.csv
All messages saved to data/raw/telegram_messages.csv
